In [1]:
import csv
import pandas as pd
import networkx as nx

In [2]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_all_only_2_or_3.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_all_only_2_or_3.csv'

In [3]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [4]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [5]:
df

,Source,Destination,Number of Tweets
0,US,US,206139.0
1,CH,CH,418.0
2,GB,GB,22922.0
3,NG,NG,2134.0
4,ZA,ZA,5623.0
...,...,...,...
1697,ES,PT,1.0
1698,PL,DK,1.0
1699,GG,US,1.0
1700,MX,MY,1.0


## Creating Graph using Networkx

In [6]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [7]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

227614.0

In [8]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [9]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,US,US,206139.0,,,,
1,CH,CH,418.0,,,,
2,GB,GB,22922.0,,,,
3,NG,NG,2134.0,,,,
4,ZA,ZA,5623.0,,,,
...,...,...,...,...,...,...,...
1697,ES,PT,1.0,,,,
1698,PL,DK,1.0,,,,
1699,GG,US,1.0,,,,
1700,MX,MY,1.0,,,,


In [10]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [11]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,US,US,206139.0,451069.0,451069.0,227614.0,223455.0
1,CH,CH,418.0,1601.0,1601.0,830.0,771.0
2,GB,GB,22922.0,62857.0,62857.0,31618.0,31239.0
3,NG,NG,2134.0,5371.0,5371.0,2620.0,2751.0
4,ZA,ZA,5623.0,13577.0,13577.0,6692.0,6885.0
...,...,...,...,...,...,...,...
1697,ES,PT,1.0,744.0,391.0,317.0,263.0
1698,PL,DK,1.0,607.0,373.0,274.0,220.0
1699,GG,US,1.0,5.0,451069.0,2.0,223455.0
1700,MX,MY,1.0,1617.0,377.0,807.0,208.0


In [12]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,US,US,206139.0,451069.0,451069.0,227614.0,223455.0,,
1,CH,CH,418.0,1601.0,1601.0,830.0,771.0,,
2,GB,GB,22922.0,62857.0,62857.0,31618.0,31239.0,,
3,NG,NG,2134.0,5371.0,5371.0,2620.0,2751.0,,
4,ZA,ZA,5623.0,13577.0,13577.0,6692.0,6885.0,,
...,...,...,...,...,...,...,...,...,...
1697,ES,PT,1.0,744.0,391.0,317.0,263.0,,
1698,PL,DK,1.0,607.0,373.0,274.0,220.0,,
1699,GG,US,1.0,5.0,451069.0,2.0,223455.0,,
1700,MX,MY,1.0,1617.0,377.0,807.0,208.0,,


In [13]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [14]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,US,US,206139.0,451069.0,451069.0,227614.0,223455.0,92.250789,90.565167
1,CH,CH,418.0,1601.0,1601.0,830.0,771.0,54.215305,50.361446
2,GB,GB,22922.0,62857.0,62857.0,31618.0,31239.0,73.376228,72.496679
3,NG,NG,2134.0,5371.0,5371.0,2620.0,2751.0,77.571792,81.450382
4,ZA,ZA,5623.0,13577.0,13577.0,6692.0,6885.0,81.670298,84.025702
...,...,...,...,...,...,...,...,...,...
1697,ES,PT,1.0,744.0,391.0,317.0,263.0,0.380228,0.315457
1698,PL,DK,1.0,607.0,373.0,274.0,220.0,0.454545,0.364964
1699,GG,US,1.0,5.0,451069.0,2.0,223455.0,0.000448,50.0
1700,MX,MY,1.0,1617.0,377.0,807.0,208.0,0.480769,0.123916


## Extensions

### Tweets per population and 2 versions of Cij 

In [15]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [16]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [17]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [18]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,US,US,206139.0,451069.0,451069.0,227614.0,223455.0,92.250789,90.565167,,,
1,CH,CH,418.0,1601.0,1601.0,830.0,771.0,54.215305,50.361446,,,
2,GB,GB,22922.0,62857.0,62857.0,31618.0,31239.0,73.376228,72.496679,,,
3,NG,NG,2134.0,5371.0,5371.0,2620.0,2751.0,77.571792,81.450382,,,
4,ZA,ZA,5623.0,13577.0,13577.0,6692.0,6885.0,81.670298,84.025702,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,ES,PT,1.0,744.0,391.0,317.0,263.0,0.380228,0.315457,,,
1698,PL,DK,1.0,607.0,373.0,274.0,220.0,0.454545,0.364964,,,
1699,GG,US,1.0,5.0,451069.0,2.0,223455.0,0.000448,50.0,,,
1700,MX,MY,1.0,1617.0,377.0,807.0,208.0,0.480769,0.123916,,,


In [19]:
import pypopulation

In [20]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [21]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,US,US,206139.0,451069.0,451069.0,227614.0,223455.0,92.250789,90.565167,0.000314,,
1,CH,CH,418.0,1601.0,1601.0,830.0,771.0,54.215305,50.361446,0.000024,,
2,GB,GB,22922.0,62857.0,62857.0,31618.0,31239.0,73.376228,72.496679,0.000171,,
3,NG,NG,2134.0,5371.0,5371.0,2620.0,2751.0,77.571792,81.450382,0.000005,,
4,ZA,ZA,5623.0,13577.0,13577.0,6692.0,6885.0,81.670298,84.025702,0.000048,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,ES,PT,1.0,744.0,391.0,317.0,263.0,0.380228,0.315457,0.0,,
1698,PL,DK,1.0,607.0,373.0,274.0,220.0,0.454545,0.364964,0.0,,
1699,GG,US,1.0,5.0,451069.0,2.0,223455.0,0.000448,50.0,0.0,,
1700,MX,MY,1.0,1617.0,377.0,807.0,208.0,0.480769,0.123916,0.0,,


In [22]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [23]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,US,US,206139.0,451069.0,451069.0,227614.0,223455.0,92.250789,90.565167,0.000314,0.228501,0.457001
1,CH,CH,418.0,1601.0,1601.0,830.0,771.0,54.215305,50.361446,0.000024,0.130543,0.261087
2,GB,GB,22922.0,62857.0,62857.0,31618.0,31239.0,73.376228,72.496679,0.000171,0.182335,0.364669
3,NG,NG,2134.0,5371.0,5371.0,2620.0,2751.0,77.571792,81.450382,0.000005,0.198659,0.397319
4,ZA,ZA,5623.0,13577.0,13577.0,6692.0,6885.0,81.670298,84.025702,0.000048,0.207078,0.414156
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,ES,PT,1.0,744.0,391.0,317.0,263.0,0.380228,0.315457,0.0,0.000881,0.001724
1698,PL,DK,1.0,607.0,373.0,274.0,220.0,0.454545,0.364964,0.0,0.00102,0.002024
1699,GG,US,1.0,5.0,451069.0,2.0,223455.0,0.000448,50.0,0.0,0.000002,0.000004
1700,MX,MY,1.0,1617.0,377.0,807.0,208.0,0.480769,0.123916,0.0,0.000502,0.000985


In [24]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_all_only_2_or_3.csv', index=False)